In [1]:
# Load jld files and analyze

import h5py
import numpy as np
import matplotlib.pyplot as plt

In [2]:
component_ids = [8,12,6,19,11,27,7,35,21,2,17,16,29,30,5]

In [3]:
components_survive_data_mean = np.zeros((len(component_ids), 100))
components_survive_data_std = np.zeros((len(component_ids), 100))


for i in range(len(component_ids)):
    component_id = component_ids[i]
    f = h5py.File('survive_data_' + str(component_id) + '.jld2', 'r')
    survive_data = f['data'][:]
    components_survive_data_mean[i,:] = np.mean(survive_data/100.0, axis=0)
    components_survive_data_std[i,:] = np.std(survive_data/100.0, axis=0)

overall_survie_data_mean = np.mean(components_survive_data_mean, axis=0)
overall_survie_data_std = np.std(components_survive_data_mean, axis=0)
budgets = list(np.arange(51)*20)

TypeError: unsupported operand type(s) for /: 'Dataset' and 'float'

In [ ]:
# scientific plot plotting the overall survival data with std band around mean

plt.figure(figsize=(10,6))
plt.plot(budgets, overall_survie_data_mean, color='black', linewidth=2)
plt.fill_between(budgets, overall_survie_data_mean - overall_survie_data_std, overall_survie_data_mean + overall_survie_data_std, color='black', alpha=0.2)
plt.xlabel('Budget')
plt.ylabel('Survival Rate')
plt.title('Overall Survival Rate')
plt.show()